<a href="https://colab.research.google.com/github/lanukahjo/Image-Splicing-Localisation/blob/main/Image_Splicing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Image Splicing Localisation
#### This notebook is the main flow of execution

In [131]:
from skimage import data
from skimage import filters
from skimage import io
from skimage.feature import greycomatrix
import numpy as np
from skimage import color
import os
import math

Reading the file from the given location
Manually for now....

In [132]:
PATH_TO_AUTHENTIC_IMG = '/content/AU_S_005.bmp'
PATH_TO_SPLICED_IMG = '/content/SP_S_005.bmp';
THRESHOLD = 0.3;
BLOCK_SIZE = 16;
STEP_SIZE = 4;

###Assumption
- Images are already in grayscale format (0-255)


In [133]:
authentic_img = io.imread(PATH_TO_AUTHENTIC_IMG);
spliced_img = io.imread(PATH_TO_SPLICED_IMG); 

In [134]:
output_img_euclidean_dist = spliced_img;
output_img_hellinger_dist = spliced_img;

authentic_img_grey_reduced = (authentic_img/32).astype(int);
spliced_img_grey_reduced = (spliced_img/32).astype(int);

img_height = authentic_img.shape[0]
img_width = authentic_img.shape[1]

In [135]:
# Get Hellinger Distance between two numpy ndarrays x and y
def getHellingerDistance(x, y):
  dot_product = np.multiply(x,y);
  squared_matrix = x - 2*np.sqrt(dot_product) + y;
  hellinger_distance = math.sqrt(np.sum(squared_matrix)/2.0); 
  return hellinger_distance;

In [136]:
# Get euclidean distance between two numpy ndarrays x and y
def getEuclideanDistance(x, y):
  euclidean_distance = math.sqrt(np.sum(np.square(x-y)))
  return euclidean_distance; 

In [137]:
for i in range(BLOCK_SIZE-1,img_height,STEP_SIZE):
  for j in range(BLOCK_SIZE-1,img_width,STEP_SIZE):
    authentic_block = authentic_img_grey_reduced[i+1-BLOCK_SIZE:i+1, j+1-BLOCK_SIZE:j+1]
    spliced_block = spliced_img_grey_reduced[i+1-BLOCK_SIZE:i+1, j+1-BLOCK_SIZE:j+1]
    
    glcm_matrix_auth = greycomatrix(authentic_block,distances=[1],angles=[0],levels = 8,normed=True)[:,:,0,0]
    glcm_matrix_spl = greycomatrix(spliced_block,distances=[1],angles=[0],levels = 8,normed=True)[:,:,0,0]
    
    hellinger_distance = getHellingerDistance(glcm_matrix_auth, glcm_matrix_spl);
    euclidean_distance = getEuclideanDistance(glcm_matrix_auth, glcm_matrix_spl);

    if (hellinger_distance >= THRESHOLD):
      colored_version = color.gray2rgb(spliced_img[:32,:32])[:,:,0]
      output_img_hellinger_dist[i+1-BLOCK_SIZE:i+1, j+1-BLOCK_SIZE:j+1] = np.maximum(
          output_img_hellinger_dist[i+1-BLOCK_SIZE:i+1, j+1-BLOCK_SIZE:j+1],
          255*np.ones((BLOCK_SIZE,BLOCK_SIZE)))
      
    if (euclidean_distance >= THRESHOLD):
      colored_version = color.gray2rgb(spliced_img[:32,:32])[:,:,0]
      output_img_euclidean_dist[i+1-BLOCK_SIZE:i+1, j+1-BLOCK_SIZE:j+1] = np.maximum(
          output_img_euclidean_dist[i+1-BLOCK_SIZE:i+1, j+1-BLOCK_SIZE:j+1],
          255*np.ones((BLOCK_SIZE,BLOCK_SIZE)))

In [138]:
io.imsave('output_euclidean.bmp', output_img_euclidean_dist)
io.imsave('output_hellinger.bmp', output_img_hellinger_dist)